In [20]:
import threading
from queue import Queue
import cv2 
from PIL import Image,ImageFont,ImageDraw
import os
from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

def create_file(filenamelist):
    for filename in filenamelist:
        img = Image.new('RGB', (125, 125),"red")
        img.save('./pic/'+str(filename)+'.jpg')


def multithread(thread_num,file_num):
    namelist = range(file_num)
    k = int(file_num/thread_num)
    list_threads = []
    for i in range(thread_num):
        list = namelist[i*k:i*k+k]
        thread  = threading.Thread(target=create_file,args=(list,))
        thread.start()
        list_threads.append(thread)
    
    for thread in list_threads:
        thread.join()


multithread(2,10)



0 5
1 5


In [1]:
import cv2 
from PIL import Image,ImageFont,ImageDraw
import os
from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import threading
def get_char(r,g,b,alpha = 256):
    ascii_char = list("#RMNHQODBWGPZ*@$C&98?32I1>!:-;. ")
    #ascii_char = list("$@B%8&WM#*oahkbdpqwmZO0QLCJUYXzcvunxrjft/\|()1{}[]?-_+~<>i!lI;:oa+>!:+. ")
    if alpha == 0:
        return ''
    length = len(ascii_char)
    gray = int(0.2126 * r + 0.7152 * g + 0.0722 * b)
    unit = (256.0+1)/len(ascii_char)
    return ascii_char[int(gray/unit)]

#将视频转换为图片 并进行计数，返回总共生成了多少张图片！
def video_to_pic(vp):
    #vp = cv2.VideoCapture(video_path)
    number = 0
    if vp.isOpened():
        r,frame = vp.read()
        if not os.path.exists('cache_pic'):
            os.mkdir('cache_pic')
        os.chdir('cache_pic')
    else:
        r = False
    while r:
        number += 1
        cv2.imwrite(str(number)+'.jpg',frame)
        r,frame = vp.read()
    print('\n由视频一共生成了{}张图片！'.format(number))
    os.chdir("..")
    return number


def img_to_char(image_path,raw_width,raw_height,task):
    width = int(raw_width/ 6)
    height = int(raw_height / 15)
    im = Image.open(image_path).convert('RGB')#必须以RGB模式打开
    im = im.resize((width,height),Image.NEAREST)
    
    txt = ''
    color = []
    for i in range(height):
        for j in range(width):
            pixel = im.getpixel((j, i))
            color.append((pixel[0],pixel[1],pixel[2])) #将颜色加入进行索引
            if len(pixel)==4 :
                txt +=get_char(pixel[0],pixel[1],pixel[2],pixel[3])
            else:
                txt +=get_char(pixel[0],pixel[1],pixel[2])
        txt += '\n'
        color.append((255,255,255))

    im_txt = Image.new("RGB",(raw_width,raw_height),(255,255,255))
    dr = ImageDraw.Draw(im_txt)
    #font = ImageFont.truetype('consola.ttf', 10, encoding='unic') #改为这个字体会让图片比例改变
    font = ImageFont.load_default().font
    x,y = 0,0
    font_w,font_h=font.getsize(txt[1])
    font_h *= 1.37 #调整字体大小
    for i in range(len(txt)):
        if(txt[i]=='\n'):
            x += font_h
            y = -font_w
        dr.text((y,x),txt[i] ,fill = color[i])
        y+=font_w
    os.chdir('cache_char')
    im_txt.save(str(task)+'.jpg')
    os.chdir("..")
    return 0



def star_to_char(number_start,number_end,save_pic_path):
    if not os.path.exists('cache_char'):
        os.mkdir('cache_char')
    img_path_list = [save_pic_path + r'/{}.jpg'.format(i) for i in range(number_start+1,number_end)] #生成目标图片文件的路径列表
    task = 0
    for image_path in img_path_list:
        img_width , img_height = Image.open(image_path).size   #获取图片的分辨率
        task += 1
        img_to_char(image_path, img_width , img_height, task)
        print('{}/{} is finished in thread {}.'.format(task,number_end-number_start,threading.currentThread().getName()))
    print('=======================')
    print('All image was finished!')
    print('=======================')
    return 0

def multithread(thread_num,file_num,save_pic_path):
    namelist = range(file_num)
    k = int(file_num/thread_num)
    list_threads = []
    for i in range(thread_num):
        list = namelist[i*k:i*k+k]
        thread  = threading.Thread(target=star_to_char,args=(i*k,i*k+k,save_pic_path))
        thread.start()
        list_threads.append(thread)
    
    for thread in list_threads:
        thread.join()

def process_bar(percent, start_str='', end_str='', total_length=0):
    #进度条
    bar = ''.join("■ " * int(percent * total_length)) + ''
    bar = '\r' + start_str + bar.ljust(total_length) + ' {:0>4.1f}%|'.format(percent*100) + end_str
    print(bar, end='', flush=True)

def jpg_to_video(char_image_path,FPS):
    video_fourcc=VideoWriter_fourcc(*"MP42")  # 设置视频编码器,这里使用使用MP42编码器,可以生成更小的视频文件
    char_img_path_list = [char_image_path + r'/{}.jpg'.format(i) for i in range(1,number+1)] #生成目标字符图片文件的路径列表
    char_img_test = Image.open(char_img_path_list[1]).size   #获取图片的分辨率
    if not os.path.exists('video'):
            os.mkdir('video')
    video_writter= VideoWriter('video/new_char_video.avi' , video_fourcc , FPS , char_img_test)
    sum  = len(char_img_path_list)
    count = 0
    for image_path in char_img_path_list:
        img = cv2.imread(image_path)
        video_writter.write(img)
        end_str = '100%'
        count= count + 1
        process_bar(count/sum, start_str='', end_str=end_str, total_length=15)
    
    video_writter.release()
    print('\n')
    print('=======================')
    print('The video is finished!')
    print('=======================')

if __name__ == '__main__':
    
    video_path = 'Slime.mp4'
    save_pic_path = 'cache_pic'
    save_charpic_path = 'cache_char'

    vp = cv2.VideoCapture(video_path)
    number = video_to_pic(vp)
    FPS = vp.get(cv2.CAP_PROP_FPS)
    multithread(5,number,save_pic_path)
    vp.release()
    jpg_to_video(save_charpic_path,FPS)

Exception in thread Thread-4:
Traceback (most recent call last):
  File "E:\Anaconda3\envs\python37\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "E:\Anaconda3\envs\python37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-962afb98ca87>", line 83, in star_to_char
    img_width , img_height = Image.open(image_path).size   #获取图片的分辨率
  File "E:\Anaconda3\envs\python37\lib\site-packages\PIL\Image.py", line 2904, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: 'cache_pic/1.jpg'

Exception in thread Thread-7:
Traceback (most recent call last):
  File "E:\Anaconda3\envs\python37\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "E:\Anaconda3\envs\python37\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1-962afb98ca87>", line 83, in star_to_char
    img_width , img_height = Image.o

FileNotFoundError: [Errno 2] No such file or directory: 'cache_char/2.jpg'